## loading in packages

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import trim, lower, regexp_replace, col, udf
from pyspark.sql.types import StringType
import nltk
import re
stop_words = set(nltk.corpus.stopwords.words('english'))

### starting spark session

In [2]:
# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Online Retail Data Cleaning") \
    .getOrCreate()

# Read the Excel file using pandas
file_path = 'online_retail_II 2.xlsx'
retail_dfs1 = pd.read_excel(file_path, sheet_name=0)
retail_dfs2 = pd.read_excel(file_path, sheet_name=1)
# Convert the pandas DataFrame to a Spark DataFrame
retails1_spark = spark.createDataFrame(retail_dfs1)
retails2_spark = spark.createDataFrame(retail_dfs2)

retails1_spark.show(5)                # Show the first 5 rows
retails1_spark.describe().show()      # Compute basic statistics for numeric columns
retails1_spark.summary().show()       # Compute summary statistics for columns
print(retails1_spark.count())  

retails2_spark.show(5)                # Show the first 5 rows
retails2_spark.describe().show()      # Compute basic statistics for numeric columns
retails2_spark.summary().show()       # Compute summary statistics for columns
print(retails2_spark.count())  


24/05/27 01:20:24 WARN Utils: Your hostname, DESKTOP-25IG4QD resolves to a loopback address: 127.0.1.1; using 172.18.229.145 instead (on interface eth0)
24/05/27 01:20:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/27 01:20:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/27 01:20:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/27 01:21:53 WARN TaskSetManager: Stage 0 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.


+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|    22041|RECORD FRAME 7" S...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
only showing top 5 rows



24/05/27 01:21:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/05/27 01:21:55 WARN TaskSetManager: Stage 1 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 01:21:57 WARN TaskSetManager: Stage 4 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+------------------+--------------------+------------------+------------------+-----------+-----------+
|summary|           Invoice|         StockCode|         Description|          Quantity|             Price|Customer ID|    Country|
+-------+------------------+------------------+--------------------+------------------+------------------+-----------+-----------+
|  count|            525461|            525461|              525461|            525461|            525461|     525461|     525461|
|   mean| 514496.9241788484| 30518.37992495103|                 NaN|10.337667305470815| 4.688834478677414|        NaN|       NULL|
| stddev|14439.209494465998|20693.168152854683|                 NaN| 107.4241102687977|146.12691395292643|        NaN|       NULL|
|    min|            489434|             10002|  DOORMAT UNION J...|             -9600|         -53594.36|    12346.0|  Australia|
|    max|           C538164|                 m|          wrong invc|             19

24/05/27 01:21:59 WARN TaskSetManager: Stage 7 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+------------------+--------------------+------------------+------------------+-----------+-----------+
|summary|           Invoice|         StockCode|         Description|          Quantity|             Price|Customer ID|    Country|
+-------+------------------+------------------+--------------------+------------------+------------------+-----------+-----------+
|  count|            525461|            525461|              525461|            525461|            525461|     525461|     525461|
|   mean| 514496.9241788484| 30518.37992495103|                 NaN|10.337667305470815| 4.688834478677414|        NaN|       NULL|
| stddev|14439.209494465998|20693.168152854683|                 NaN| 107.4241102687977|146.12691395292643|        NaN|       NULL|
|    min|            489434|             10002|  DOORMAT UNION J...|             -9600|         -53594.36|    12346.0|  Australia|
|    25%|          501879.0|           21528.0|                 NaN|               

24/05/27 01:22:00 WARN TaskSetManager: Stage 10 contains a task of very large size (2836 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 01:22:00 WARN TaskSetManager: Stage 11 contains a task of very large size (2836 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 01:22:01 WARN TaskSetManager: Stage 14 contains a task of very large size (2836 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----------------+------------------+--------------------+------------------+-----------------+-----------+-----------+
|summary|          Invoice|         StockCode|         Description|          Quantity|            Price|Customer ID|    Country|
+-------+-----------------+------------------+--------------------+------------------+-----------------+-----------+-----------+
|  count|           541910|            541910|              541910|            541910|           541910|     541910|     541910|
|   mean|559965.7926209917|27623.240210938104|                 NaN| 9.552233765754462|4.611138332934094|        NaN|       NULL|
| stddev|13428.43735533339| 16799.73762842769|                 NaN|218.08095694392486|96.75976549366531|        NaN|       NULL|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06|    12346.0|  Australia|
|    max|          C581569|                 m|   wrongly sold sets|             80995|          3

24/05/27 01:22:03 WARN TaskSetManager: Stage 17 contains a task of very large size (2836 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----------------+------------------+--------------------+------------------+-----------------+-----------+-----------+
|summary|          Invoice|         StockCode|         Description|          Quantity|            Price|Customer ID|    Country|
+-------+-----------------+------------------+--------------------+------------------+-----------------+-----------+-----------+
|  count|           541910|            541910|              541910|            541910|           541910|     541910|     541910|
|   mean|559965.7926209917|27623.240210938104|                 NaN| 9.552233765754462|4.611138332934094|        NaN|       NULL|
| stddev|13428.43735533339| 16799.73762842769|                 NaN|218.08095694392486|96.75976549366531|        NaN|       NULL|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06|    12346.0|  Australia|
|    25%|         547906.0|           21929.0|                 NaN|                 1|           

In [3]:
# Define a function to remove stopwords
def remove_stopwords(text):
    other_words = {'set', 'of','/', '-',"'s", '"s','cm','m','.','s','and','to'}
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in words if word.lower() not in other_words]
    rejoined_words = ' '.join(filtered_words)
    rejoined_words = re.sub(r's/(\d+)', r'\1', rejoined_words)
    rejoined_words = re.sub(r'set/(\d+)', r'\1', rejoined_words)

    return rejoined_words

# Register the function as a UDF
remove_stopwords_udf = udf(remove_stopwords, StringType())

# Data Cleaning Steps
def clean_df(spark_df):
    print(spark_df.count())
    # Handle missing values
    clean_spark_df = spark_df.dropna()
    print(clean_spark_df.count())

    # Remove duplicate rows
    clean_spark_df = clean_spark_df.dropDuplicates()
    print(clean_spark_df.count())
    
    # Correct data types
    clean_spark_df = clean_spark_df.withColumn("InvoiceDate", col("InvoiceDate").cast("timestamp"))
    clean_spark_df = clean_spark_df.withColumn("Customer ID", col("Customer ID").cast("integer"))

    # Remove negative or zero quantities and prices
    clean_spark_df = clean_spark_df.filter((col("Quantity") > 0) & (col("Price") > 0))
    print(clean_spark_df.count())

    # Standardize text data in 'Description'
    clean_spark_df = clean_spark_df.withColumn("Description", trim(lower(col("Description"))))
    print(clean_spark_df.count())
    
    clean_spark_df = clean_spark_df.withColumn("Description", regexp_replace(col("Description"), ' [^a-zA-Z] ', ''))
    clean_spark_df = clean_spark_df.withColumn("Description", remove_stopwords_udf(col("Description")))
    # clean_spark_df = clean_spark_df.withColumn("Description", regexp_replace(col("Description"), '[0-9]',''))
    # Ensure consistency in Customer IDs (e.g., remove anomalies)
    clean_spark_df = clean_spark_df.filter(col("Customer ID").isNotNull() & (col("Customer ID") > 0))
    print(clean_spark_df.count())

    # Remove invoices that start with C as they are cancelled
    clean_spark_df = clean_spark_df.filter(~col("Invoice").startswith("C"))

    return clean_spark_df


clean_retails1_spark = clean_df(retails1_spark)
clean_retails2_spark = clean_df(retails2_spark)

print(clean_retails1_spark.count(), clean_retails2_spark.count())
# save clean data
output_path = 'online_retail_IIs1_clean.parquet'
clean_retails1_spark.write.mode("overwrite").parquet(output_path)

output_path = 'online_retail_IIs2_clean.parquet'
clean_retails2_spark.write.mode("overwrite").parquet(output_path)

# Stop the Spark session
spark.stop()

24/05/27 01:22:03 WARN TaskSetManager: Stage 20 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.


525461


24/05/27 01:22:03 WARN TaskSetManager: Stage 23 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.


417534


24/05/27 01:22:03 WARN TaskSetManager: Stage 26 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 01:22:05 WARN TaskSetManager: Stage 32 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.


...

24/05/27 01:22:08 WARN TaskSetManager: Stage 74 contains a task of very large size (2836 KiB). The maximum recommended task size is 1000 KiB.


392693


24/05/27 01:22:09 WARN TaskSetManager: Stage 80 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 01:22:10 WARN TaskSetManager: Stage 86 contains a task of very large size (2836 KiB). The maximum recommended task size is 1000 KiB.


400916 392693


24/05/27 01:22:10 WARN TaskSetManager: Stage 92 contains a task of very large size (2746 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 01:22:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/05/27 01:22:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/05/27 01:22:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/05/27 01:22:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/05/27 01:22:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/05/27 01:22:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054